In [2]:
import pandas as pd
from scipy import stats
import numpy as np
import statsmodels.api as sm
from statsmodels.multivariate.manova import MANOVA
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

# 设置 Matplotlib 支持中文显示
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False
alpha = 0.05 # 统一设置显著性水平

# --- 第4题：跑步数据分析 ---
print("--- 第4题：跑步数据分析 ---")
# 读取2012年的跑步数据
running_data_2012 = pd.read_csv('run.csv', header=None, names=['value'], encoding='gbk')

# --- 第一问：检验样本是否来自正态分布 ---
print("\n--- 第一问：正态性检验 ---")
# 使用Shapiro-Wilk检验
shapiro_stat, shapiro_p = stats.shapiro(running_data_2012['value'])
print(f"Shapiro-Wilk检验统计量: {shapiro_stat:.4f}")
print(f"Shapiro-Wilk检验P值: {shapiro_p:.4f}")

# 根据P值判断正态性
if shapiro_p > alpha:
    print(f"结论: 在显著性水平 {alpha} 下，P值大于 {alpha}，我们不能拒绝原假设，可以认为数据样本服从正态分布。")
else:
    print(f"结论: 在显著性水平 {alpha} 下，P值小于等于 {alpha}，我们拒绝原假设，认为数据样本不服从正态分布。")


# --- 第二问：相较于2009年(平均103.5分钟)，2012年参与者平均用时是否更短 ---
print("\n--- 第二问：单样本单尾T检验 ---")
# 已知2009年的平均用时
mean_2009 = 103.5

# 执行单样本T检验，检验2012年的平均用时是否“小于”2009年的平均用时
# 因此我们使用 alternative='less'
t_statistic, t_p_value = stats.ttest_1samp(running_data_2012['value'], mean_2009, alternative='less')

print(f"检验目标: 2012年平均用时 < {mean_2009} 分钟")
print(f"T统计量: {t_statistic:.4f}")
print(f"P值 (单尾): {t_p_value:.4f}")

# 根据P值做出判断
if t_p_value < alpha:
    print(f"结论: 在显著性水平 {alpha} 下，P值小于 {alpha}，我们拒绝原假设。")
    print("有足够的统计证据表明，2012年参与者的平均用时显著短于2009年。")
else:
    print(f"结论: 在显著性水平 {alpha} 下，P值大于等于 {alpha}，我们不能拒绝原假设。")
    print("没有足够的统计证据表明2012年参与者的平均用时显著短于2009年。")


--- 第4题：跑步数据分析 ---

--- 第一问：正态性检验 ---
Shapiro-Wilk检验统计量: 0.9880
Shapiro-Wilk检验P值: 0.2233
结论: 在显著性水平 0.05 下，P值大于 0.05，我们不能拒绝原假设，可以认为数据样本服从正态分布。

--- 第二问：单样本单尾T检验 ---
检验目标: 2012年平均用时 < 103.5 分钟
T统计量: -47.1414
P值 (单尾): 0.0000
结论: 在显著性水平 0.05 下，P值小于 0.05，我们拒绝原假设。
有足够的统计证据表明，2012年参与者的平均用时显著短于2009年。


In [18]:
import pandas as pd
from scipy import stats
import numpy as np
from statsmodels.stats.multicomp import pairwise_tukeyhsd

# 读取数据集，设置表头行为 1
df = pd.read_csv('cost.csv', encoding='GB2312', header=1)

# 从第 2 行(index=1)到第 31 行(index=30)加载数据
df = df.loc[1:30]

# 重置索引
df = df.reset_index(drop=True)

# 设置列名
df.columns = ['地区', '食品(X1)', '衣着(X2)', '居住(X3)', '家庭设备用品及服务(X4)', '医疗保健(X5)',
              '交通和通信(X6)', '教育文化娱乐服务(X7)', '杂项商品和服务(X8)']

# (1) 对上述八个指标的均值和协方差进行估计
# 用到的估计量：样本均值和样本协方差矩阵
# 性质：样本均值是总体均值的无偏估计，样本协方差矩阵是总体协方差矩阵的无偏估计
mean_values = df[['食品(X1)', '衣着(X2)', '居住(X3)', '家庭设备用品及服务(X4)', '医疗保健(X5)',
                  '交通和通信(X6)', '教育文化娱乐服务(X7)', '杂项商品和服务(X8)']].mean()
cov_matrix = df[['食品(X1)', '衣着(X2)', '居住(X3)', '家庭设备用品及服务(X4)', '医疗保健(X5)',
                 '交通和通信(X6)', '教育文化娱乐服务(X7)', '杂项商品和服务(X8)']].cov()

print('各指标均值：')
print(mean_values)
print('各指标协方差矩阵：')
print(cov_matrix)
import pandas as pd
from scipy import stats
import numpy as np
from statsmodels.stats.multicomp import pairwise_tukeyhsd

# 读取数据集，设置表头行为 1
df = pd.read_csv('cost.csv', encoding='GB2312', header=1)

# 从第 2 行(index=1)到第 31 行(index=30)加载数据
df = df.loc[1:30]

# 重置索引
df = df.reset_index(drop=True)

# 设置列名
df.columns = ['地区', '食品(X1)', '衣着(X2)', '居住(X3)', '家庭设备用品及服务(X4)', '医疗保健(X5)',
              '交通和通信(X6)', '教育文化娱乐服务(X7)', '杂项商品和服务(X8)']

# (2) 对数据的正态性进行评估（α=0.01）
# 选择 Shapiro - Wilk 检验，因为它在样本量较小时（通常 n < 50）对正态性检验效果较好
alpha = 0.01
normal_results = {}
for column in ['食品(X1)', '衣着(X2)', '居住(X3)', '家庭设备用品及服务(X4)', '医疗保健(X5)',
               '交通和通信(X6)', '教育文化娱乐服务(X7)', '杂项商品和服务(X8)']:
    statistic, p_value = stats.shapiro(df[column])
    normal_results[column] = {
        '统计量': statistic,
        'p 值': p_value,
        '是否正态分布': p_value > alpha
    }

print('各指标正态性检验结果：')
for column, result in normal_results.items():
    print(f"指标：{column}")
    print(f"统计量：{result['统计量']}")
    print(f"p 值：{result['p 值']}")
    print(f"是否正态分布：{result['是否正态分布']}")
    print("-" * 30)

# (3) 按照区域对地区进行划分
# 定义地区所属区域
north = ['北京', '天津', '河北', '山西', '内蒙古']
central_east = ['辽宁', '吉林', '黑龙江', '上海', '江苏', '浙江', '安徽', '福建', '江西', '山东', '河南']
south = ['湖北', '湖南', '广东', '广西', '海南']
west = ['重庆', '四川', '贵州', '云南', '西藏', '陕西', '甘肃', '青海', '宁夏', '新疆']


# 增加所属区域标签
def get_region(area):
    if area in north:
        return '北部地区'
    elif area in central_east:
        return '中东部地区'
    elif area in south:
        return '南部地区'
    elif area in west:
        return '西部地区'


df['所属区域'] = df['地区'].apply(get_region)

# 选择符合正态分布的指标
normal_columns = [column for column, result in normal_results.items() if result.get('是否正态分布')]

# 比较四个总体的消费支出是否存在显著差异
anova_results = {}
for column in normal_columns:
    groups = df.groupby('所属区域')[column].apply(list)
    statistic, p_value = stats.f_oneway(*groups)
    anova_results[column] = {
        '统计量': statistic,
        'p 值': p_value,
        '是否有显著差异': p_value < alpha
    }

print('各符合正态分布指标的 ANOVA 检验结果：')
for column, result in anova_results.items():
    print(f"指标：{column}")
    print(f"统计量：{result['统计量']}")
    print(f"p 值：{result['p 值']}")
    print(f"是否有显著差异：{result['是否有显著差异']}")
    print("-" * 30)

# 如果存在显著差异，找出来自于哪些总体和哪些指标之间的差异
if any([result['是否有显著差异'] for result in anova_results.values()]):
    for column in normal_columns:
        if anova_results[column]['是否有显著差异']:
            tukey_results = pairwise_tukeyhsd(df[column], df['所属区域'], alpha=alpha)
            print(f'指标 {column} 的 Tukey 检验结果：')
            print(tukey_results)

各指标均值：
食品(X1)           4056.629667
衣着(X2)           1121.982333
居住(X3)           1067.655667
家庭设备用品及服务(X4)     631.797667
医疗保健(X5)          729.627333
交通和通信(X6)        1260.971333
教育文化娱乐服务(X7)     1205.070000
杂项商品和服务(X8)       389.976000
dtype: float64
各指标协方差矩阵：
                      食品(X1)        衣着(X2)         居住(X3)  家庭设备用品及服务(X4)   
食品(X1)         868349.615700  43077.706987  158073.773743  113225.894161  \
衣着(X2)          43077.706987  50612.002756   20877.596228   15482.512468   
居住(X3)         158073.773743  20877.596228   70875.214922   33774.246555   
家庭设备用品及服务(X4)  113225.894161  15482.512468   33774.246555   30439.951881   
医疗保健(X5)        20893.412189  22837.159769   30081.407412   13072.465304   
交通和通信(X6)      499556.051100  44918.539914  119601.325409   77070.259783   
教育文化娱乐服务(X7)   360423.865234  54636.030300  100892.039562   73447.014200   
杂项商品和服务(X8)    101915.578247  18009.534886   27241.496468   18714.149401   

                   医疗保健(X5)      交通和通信(X6)   教育文化娱乐

In [26]:
import pandas as pd
from scipy import stats


def sales_anova(file_path='sale.csv'):
    """
    对销售数据进行单因素方差分析，以检验不同销售方式对总销售额的影响。

    参数:
    file_path (str): 包含销售数据的CSV文件的路径。

    返回:
    tuple: 包含 F 统计量和 p 值。
    """
    try:
        # 使用 'GB2312' 编码加载数据
        df = pd.read_csv(file_path, encoding='GB2312')

        # 检查必要的列是否存在
        required_columns = ['x1', 'x2', 'x3', 'x4', '销售方式']
        if not all(col in df.columns for col in required_columns):
            raise ValueError("数据中缺少必要的列")

        # 检查是否有缺失值
        if df.isnull().any().any():
            df = df.dropna()

        # 计算每个样本的总销售额
        df['total_sales'] = df[['x1', 'x2', 'x3', 'x4']].sum(axis=1)

        # 根据销售方式分组
        sales_method_1 = df[df['销售方式'] == 1]['total_sales']
        sales_method_2 = df[df['销售方式'] == 2]['total_sales']
        sales_method_3 = df[df['销售方式'] == 3]['total_sales']

        # 验证正态性（Shapiro - Wilk 检验）
        for method, data in zip(['销售方式1', '销售方式2', '销售方式3'],
                                [sales_method_1, sales_method_2, sales_method_3]):
            _, shapiro_p = stats.shapiro(data)
            if shapiro_p < 0.05:
                print(f'{method} 可能不服从正态分布，方差分析结果可能不准确。')

        # 验证方差齐性（Levene 检验）
        _, levene_p = stats.levene(sales_method_1, sales_method_2, sales_method_3)
        if levene_p < 0.05:
            print('各销售方式的销售额方差可能不齐，方差分析结果可能不准确。')

        # 执行单因素方差分析
        f_statistic, p_value = stats.f_oneway(sales_method_1, sales_method_2, sales_method_3)

        return f_statistic, p_value
    except FileNotFoundError:
        print(f"文件 {file_path} 未找到")
        return None, None
    except ValueError as e:
        print(e)
        return None, None


# 执行函数并打印结果
f_stat, p_val = sales_anova('sale.csv')

if f_stat is not None and p_val is not None:
    print(f"F-statistic: {f_stat}")
    print(f"P-value: {p_val}")

    # 解释结果
    alpha = 0.05  # 显著性水平
    if p_val < alpha:
        print("\n由于 p-value 小于 0.05，我们拒绝原假设。")
        print("结论：三种销售方式的效果有显著性差异。")
    else:
        print("\n由于 p-value 大于或等于 0.05，我们无法拒绝原假设。")
        print("结论：没有足够的证据表明三种销售方式的效果有显著性差异。")

F-statistic: 5.609158037185223
P-value: 0.01278073707025498

由于 p-value 小于 0.05，我们拒绝原假设。
结论：三种销售方式的效果有显著性差异。


In [31]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from scipy import stats

# 用于正常显示中文标签
matplotlib.rcParams['font.sans-serif'] = ['SimHei']
matplotlib.rcParams['axes.unicode_minus'] = False

def profile_analysis(file_path='marry.csv'):
    """
    对婚姻调查数据进行完整的轮廓分析。
    """
    # --- 1. 加载和准备数据 ---
    try:
        df = pd.read_csv(file_path, encoding='gbk')
    except Exception:
        df = pd.read_csv(file_path, encoding='utf-8')

    husbands = df[df['总体'] == 1].iloc[:, :4]
    wives = df[df['总体'] == 2].iloc[:, :4]
    mean_husbands = husbands.mean()
    mean_wives = wives.mean()

    # --- 2. (1) 绘制轮廓图 ---
    plt.figure(figsize=(10, 6))
    questions = ['问题1: 贡献度', '问题2: 结果', '问题3: 爱情热度', '问题4: 可结伴度']
    plt.plot(questions, mean_husbands, marker='o', linestyle='-', label='丈夫 (Husbands)')
    plt.plot(questions, mean_wives, marker='s', linestyle='--', label='妻子 (Wives)')
    plt.title('丈夫与妻子的打分轮廓图 (Profile Plot)')
    plt.xlabel('调查问题')
    plt.ylabel('平均分 (Mean Score)')
    plt.grid(True)
    plt.legend()
    plt.ylim(3.5, 5.5)
    
    # ==========================================================
    # == 新增代码：修正X轴标签超出边界的问题 ==
    plt.xticks(rotation=30, ha='right') # 将标签旋转30度
    plt.tight_layout()                  # 自动调整布局，防止标签被裁切
    # ==========================================================
    
    # 保存图像文件
    plot_filename = 'profile_plot.png'
    plt.savefig(plot_filename)
    plt.close()

    print(f"--- 轮廓分析结果 ---\n")
    print(f"(1) 轮廓图已生成并保存为: {plot_filename} (已修正边界问题)")

    # --- 3. 执行统计检验 (这部分代码与之前相同) ---
    n1, n2 = len(husbands), len(wives)
    p = len(husbands.columns)

    # a. 平行性检验
    C = np.array([[1, -1, 0, 0], [0, 1, -1, 0], [0, 0, 1, -1]])
    husbands_transformed = husbands.values @ C.T
    wives_transformed = wives.values @ C.T
    mean_h_trans = husbands_transformed.mean(axis=0)
    mean_w_trans = wives_transformed.mean(axis=0)
    cov_h_trans = np.cov(husbands_transformed, rowvar=False)
    cov_w_trans = np.cov(wives_transformed, rowvar=False)
    Sp_trans = ((n1 - 1) * cov_h_trans + (n2 - 1) * cov_w_trans) / (n1 + n2 - 2)
    diff_means_trans = mean_h_trans - mean_w_trans
    T2_parallelism = (n1 * n2) / (n1 + n2) * diff_means_trans.T @ np.linalg.inv(Sp_trans) @ diff_means_trans
    q = p - 1
    F_parallelism = T2_parallelism * (n1 + n2 - q - 1) / (q * (n1 + n2 - 2))
    p_value_parallelism = 1 - stats.f.cdf(F_parallelism, q, n1 + n2 - q - 1)

    print("\n(2) 轮廓性质检验:")
    print("\n--- a. 平行性检验 (Test for Parallelism) ---")
    print("原假设 (H0p): 丈夫和妻子的打分轮廓是平行的。")
    print(f"   F-statistic = {F_parallelism:.4f}")
    print(f"   P-value = {p_value_parallelism:.4f}")
    if p_value_parallelism < 0.05:
        print("   结论: P-value < 0.05，拒绝原假设。两个轮廓不平行。")
        print("\n由于轮廓不平行，后续的重合性与水平性检验不再有意义。")
        return
    else:
        print("   结论: P-value >= 0.05，不拒绝原假设。可以认为两个轮廓是平行的。")

    # b. 重合性检验
    sum_husbands = husbands.sum(axis=1)
    sum_wives = wives.sum(axis=1)
    _, p_value_coincidence = stats.ttest_ind(sum_husbands, sum_wives)
    
    print("\n--- b. 重合性检验 (Test for Coincidence) ---")
    print("原假设 (H0c): 在平行的前提下，两个轮廓是重合的。")
    print(f"   P-value = {p_value_coincidence:.4f}")
    if p_value_coincidence < 0.05:
        print("   结论: P-value < 0.05，拒绝原假设。两个轮廓不重合。")
        print("\n由于轮廓不重合，水平性检验将不再有意义。")
        return
    else:
        print("   结论: P-value >= 0.05，不拒绝原假设。可以认为两个轮廓是重合的。")

    # c. 水平性检验
    pooled_transformed = np.vstack((husbands_transformed, wives_transformed))
    mean_pooled_trans = pooled_transformed.mean(axis=0)
    cov_pooled_trans = np.cov(pooled_transformed, rowvar=False)
    n_total = n1 + n2
    T2_flatness = n_total * mean_pooled_trans.T @ np.linalg.inv(cov_pooled_trans) @ mean_pooled_trans
    F_flatness = T2_flatness * (n_total - q) / (q * (n_total - 1))
    p_value_flatness = 1 - stats.f.cdf(F_flatness, q, n_total - q)

    print("\n(3) 水平性检验 (Test for Flatness):")
    print("原假设 (H0f): 在平行且重合的前提下，轮廓是水平的。")
    print(f"   F-statistic = {F_flatness:.4f}")
    print(f"   P-value = {p_value_flatness:.4f}")
    if p_value_flatness < 0.05:
        print("   结论: P-value < 0.05，拒绝原假设。轮廓不是水平的（即对不同问题的打分有差异）。")
    else:
        print("   结论: P-value >= 0.05，不拒绝原假设。可以认为轮廓是水平的（即对不同问题的打分无显著差异）。")


# 运行分析
if __name__ == '__main__':
    profile_analysis('marry.csv')

--- 轮廓分析结果 ---

(1) 轮廓图已生成并保存为: profile_plot.png (已修正边界问题)

(2) 轮廓性质检验:

--- a. 平行性检验 (Test for Parallelism) ---
原假设 (H0p): 丈夫和妻子的打分轮廓是平行的。
   F-statistic = 2.5799
   P-value = 0.0626
   结论: P-value >= 0.05，不拒绝原假设。可以认为两个轮廓是平行的。

--- b. 重合性检验 (Test for Coincidence) ---
原假设 (H0c): 在平行的前提下，两个轮廓是重合的。
   P-value = 0.2207
   结论: P-value >= 0.05，不拒绝原假设。可以认为两个轮廓是重合的。

(3) 水平性检验 (Test for Flatness):
原假设 (H0f): 在平行且重合的前提下，轮廓是水平的。
   F-statistic = 7.9931
   P-value = 0.0002
   结论: P-value < 0.05，拒绝原假设。轮廓不是水平的（即对不同问题的打分有差异）。
